In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
cars=pd.read_csv("CarPrice_Assignment.csv")

# Agenda: Geely Auto wants to enter USA Market
# 1. Factors affecting pricing of cars
# 2. Which variables are significant in 
#    predicting the price of a car,
# 3. How Well those variables describing price
# sns.pairplot(cars)


In [3]:

# Catgorical Variables
# 1. Fueltype => ['gas', 'diesel']
# 2. Aspiration =>['std', 'turbo']
# 3. door number =>['two', 'four']
# 4. carboday =>['convertible', 'hatchback', 'sedan', 'wagon', 'hardtop']
# 5  drive wheel =>['rwd', 'fwd', '4wd']
# 6  engine location =>['front', 'rear']
# 7. fuelsystem =>['mpfi', '2bbl', 'mfi', '1bbl', 'spfi', '4bbl', 'idi', 'spdi']
# 8. engine type =>['dohc', 'ohcv', 'ohc', 'l', 'rotor', 'ohcf', 'dohcv']
# 9. cylindernumbers =>['four', 'six', 'five', 'three', 'twelve', 'two', 'eight']
# cars.fueltype.describe()
cars["fueltype"]=cars["fueltype"].map({"gas":1,"diesel":0})
cars.aspiration=cars["aspiration"].map({"std":1,"turbo":0})
cars.doornumber=cars["doornumber"].map({"two":1,"four":0})
cars.enginelocation=cars.enginelocation.map({"front":1,"rear":0})

# cars["sedan"]=cars.carbody.apply(lambda s:1 if s=="sedan" else 0)
# cars["hatchback"]=cars.carbody.apply(lambda s:1 if s=="hatchback" else 0)
# cars["convertible"]=cars.carbody.apply(lambda s:1 if s=="convertible" else 0)
# cars["wagon"]=cars.carbody.apply(lambda s:1 if s=="wagon" else 0)



carbody=pd.get_dummies(cars.carbody,drop_first=True)
drive_wheel=pd.get_dummies(cars.drivewheel,drop_first=True)
fuelsystem=pd.get_dummies(cars.fuelsystem,drop_first=True)
enginetype=pd.get_dummies(cars.enginetype,drop_first=True)
cylindernumbers=pd.get_dummies(cars.cylindernumber,drop_first=True)
#Adding the results to the master dataframe
cars = pd.concat([cars,carbody,drive_wheel,fuelsystem,enginetype,cylindernumbers],axis=1)

In [4]:
cars.drop(["carbody","car_ID","drivewheel","fuelsystem","enginetype","cylindernumber","CarName"],axis=1,inplace=True)

In [5]:
#defining a normalisation function 
def normalize (x): 
    return ( (x-np.mean(x))/ (max(x) - min(x)))
                                            
                                              
# applying normalize ( ) to all columns 
cars = cars.apply(normalize) 



In [6]:
col=list(cars.columns)

# X=cars[['symboling','fueltype','aspiration','doornumber','enginelocation','wheelbase','carlength','carwidth',
#  'carheight','curbweight','enginesize','boreratio','stroke','compressionratio','horsepower','peakrpm',
#  'citympg','highwaympg','convertible','hardtop','hatchback','sedan','wagon','4wd','fwd','rwd','1bbl','2bbl',
#  '4bbl','idi','mfi','mpfi','spdi','spfi','dohc','dohcv','l','ohc','ohcf','ohcv','rotor','eight','five','four','six',
#  'three','twelve','two']]
# y=cars["price"]

x_vars=['symboling',
 'fueltype',
 'aspiration',
 'doornumber',
 'enginelocation',
 'wheelbase',
 'carlength',
 'carwidth',
 'carheight',
 'curbweight',
 'enginesize',
 'boreratio',
 'stroke',
 'compressionratio',
 'horsepower',
 'peakrpm',
 'citympg',
 'highwaympg',
 'hardtop',
 'hatchback',
 'sedan',
 'wagon',
 'fwd',
 'rwd',
 '2bbl',
 '4bbl',
 'idi',
 'mfi',
 'mpfi',
 'spdi',
 'spfi',
 'dohcv',
 'l',
 'ohc',
 'ohcf',
 'ohcv',
 'rotor',
 'five',
 'four',
 'six',
 'three',
 'twelve',
 'two']
y=cars["price"]
col.pop(col.index("price"))
X=cars[col]







In [7]:
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, train_size=0.7,test_size=0.3,random_state=100)


/home/abhi/venvs/notebook/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_1 = sm.OLS(y,X).fit()
print (lm_1.summary())
# ['mpfi', '2bbl', 'mfi', '1bbl', 'spfi', '4bbl', 'idi']

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     61.79
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           4.86e-80
Time:                        23:27:08   Log-Likelihood:                 328.88
No. Observations:                 205   AIC:                            -573.8
Df Residuals:                     163   BIC:                            -434.2
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             8.522e-17      0.004  

In [9]:
# UDF for calculating vif value
def vif_cal(input_data, dependent_col):
    vif_df = pd.DataFrame( columns = ['Var', 'Vif'])
    x_vars=input_data.drop([dependent_col], axis=1)
    xvar_names=x_vars.columns
    for i in range(0,xvar_names.shape[0]):
        y=x_vars[xvar_names[i]] 
        x=x_vars[xvar_names.drop(xvar_names[i])]
        rsq=sm.OLS(y,x).fit().rsquared  
        vif=round(1/(1-rsq),2)
        vif_df.loc[i] = [xvar_names[i], vif]
    return vif_df.sort_values(by = 'Vif', axis=0, ascending=False, inplace=False)

In [10]:
# vif_cal(input_data=cars, dependent_col="price")
cars

,symboling,fueltype,aspiration,doornumber,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,...,ohc,ohcf,ohcv,rotor,five,four,six,three,twelve,two
0,0.433171,0.097561,0.180488,0.560976,0.014634,-0.296110,-0.078347,-0.150650,-0.410407,-0.002935,...,-0.721951,-0.073171,-0.063415,-0.019512,-0.053659,0.22439,-0.117073,-0.004878,-0.004878,-0.019512
1,0.433171,0.097561,0.180488,0.560976,0.014634,-0.296110,-0.078347,-0.150650,-0.410407,-0.002935,...,-0.721951,-0.073171,-0.063415,-0.019512,-0.053659,0.22439,-0.117073,-0.004878,-0.004878,-0.019512
2,0.033171,0.097561,0.180488,0.560976,0.014634,-0.124099,-0.042526,-0.033984,-0.110407,0.103737,...,-0.721951,-0.073171,0.936585,-0.019512,-0.053659,-0.77561,0.882927,-0.004878,-0.004878,-0.019512
3,0.233171,0.097561,0.180488,-0.439024,0.014634,0.030420,0.038071,0.024350,0.047927,-0.084781,...,0.278049,-0.073171,-0.063415,-0.019512,-0.053659,0.22439,-0.117073,-0.004878,-0.004878,-0.019512
4,0.233171,0.097561,0.180488,-0.439024,0.014634,0.018758,0.038071,0.041016,0.047927,0.104125,...,0.278049,-0.073171,-0.063415,-0.019512,0.946341,-0.77561,-0.117073,-0.004878,-0.004878,-0.019512
5,0.233171,0.097561,0.180488,0.560976,0.014634,0.030420,0.048518,0.032683,-0.052073,-0.018839,...,0.278049,-0.073171,-0.063415,-0.019512,0.946341,-0.77561,-0.117073,-0.004878,-0.004878,-0.019512
6,0.033171,0.097561,0.180488,-0.439024,0.014634,0.205347,0.278369,0.457683,0.164593,0.111883,...,0.278049,-0.073171,-0.063415,-0.019512,0.946341,-0.77561,-0.117073,-0.004878,-0.004878,-0.019512
7,0.033171,0.097561,0.180488,-0.439024,0.014634,0.205347,0.278369,0.457683,0.164593,0.154552,...,0.278049,-0.073171,-0.063415,-0.019512,0.946341,-0.77561,-0.117073,-0.004878,-0.004878,-0.019512
8,0.033171,0.097561,-0.819512,-0.439024,0.014634,0.205347,0.278369,0.457683,0.181260,0.205754,...,0.278049,-0.073171,-0.063415,-0.019512,0.946341,-0.77561,-0.117073,-0.004878,-0.004878,-0.019512
9,-0.166829,0.097561,-0.819512,0.560976,0.014634,0.021674,0.061951,0.166016,-0.143740,0.192954,...,0.278049,-0.073171,-0.063415,-0.019512,0.946341,-0.77561,-0.117073,-0.004878,-0.004878,-0.019512


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [12]:
X.drop(["symboling"],axis=1,inplace=True)

In [13]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_2 = sm.OLS(y,X).fit()
print (lm_2.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     63.68
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           6.33e-81
Time:                        23:27:09   Log-Likelihood:                 328.82
No. Observations:                 205   AIC:                            -575.6
Df Residuals:                     164   BIC:                            -439.4
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             9.693e-17      0.004  

In [14]:
vif_cal(input_data=cars, dependent_col="price")

/home/abhi/venvs/notebook/lib/python2.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


,Var,Vif
42,two,inf
36,rotor,inf
26,idi,inf
1,fueltype,inf
13,compressionratio,184.330000
38,four,68.930000
10,enginesize,49.510000
16,citympg,39.270000
17,highwaympg,36.370000
9,curbweight,33.680000


In [15]:
X.drop(["fueltype"],axis=1,inplace=True)
cars.drop(["fueltype"],axis=1,inplace=True)

In [16]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_3 = sm.OLS(y,X).fit()
print (lm_3.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     63.68
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           6.33e-81
Time:                        23:27:11   Log-Likelihood:                 328.82
No. Observations:                 205   AIC:                            -575.6
Df Residuals:                     164   BIC:                            -439.4
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             9.823e-17      0.004  

In [17]:

vif_cal(input_data=cars, dependent_col="price")


/home/abhi/venvs/notebook/lib/python2.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


,Var,Vif
41,two,inf
35,rotor,inf
12,compressionratio,184.330000
25,idi,184.030000
37,four,68.930000
9,enginesize,49.510000
15,citympg,39.270000
16,highwaympg,36.370000
8,curbweight,33.680000
13,horsepower,32.770000


In [18]:

drop_col=["two","rotor"]

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [19]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_3 = sm.OLS(y,X).fit()
print (lm_3.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     65.71
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           7.74e-82
Time:                        23:27:11   Log-Likelihood:                 328.82
No. Observations:                 205   AIC:                            -577.6
Df Residuals:                     165   BIC:                            -444.7
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             9.671e-17      0.004  

In [20]:
drop_col=["compressionratio","idi"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
12,compressionratio,184.31
25,idi,183.87
15,citympg,38.48
16,highwaympg,35.97
8,curbweight,32.16
13,horsepower,31.96
9,enginesize,27.26
36,four,23.80
19,sedan,18.59
5,carlength,15.21


In [21]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [22]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     68.69
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           5.31e-83
Time:                        23:27:12   Log-Likelihood:                 326.87
No. Observations:                 205   AIC:                            -577.7
Df Residuals:                     167   BIC:                            -451.5
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.529e-16      0.004   4.02e

In [23]:
drop_col=["citympg"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
14,citympg,36.18
15,highwaympg,35.20
8,curbweight,30.49
12,horsepower,28.00
9,enginesize,24.49
34,four,23.56
18,sedan,18.54
5,carlength,14.59
4,wheelbase,14.24
17,hatchback,14.04


In [24]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [25]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     70.13
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           1.67e-83
Time:                        23:27:13   Log-Likelihood:                 325.65
No. Observations:                 205   AIC:                            -577.3
Df Residuals:                     168   BIC:                            -454.3
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           1.323e-16      0.004   3.47e

In [26]:
drop_col=["doornumber","curbweight"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
8,curbweight,30.13
12,horsepower,26.67
33,four,23.56
9,enginesize,23.38
17,sedan,18.52
5,carlength,14.39
4,wheelbase,14.14
16,hatchback,14.01
20,rwd,13.51
19,fwd,10.65


In [27]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [28]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.935
Model:                            OLS   Adj. R-squared:                  0.922
Method:                 Least Squares   F-statistic:                     72.11
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           5.62e-84
Time:                        23:27:14   Log-Likelihood:                 321.70
No. Observations:                 205   AIC:                            -573.4
Df Residuals:                     170   BIC:                            -457.1
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           8.717e-17      0.004   2.26e

In [29]:
drop_col=["horsepower","boreratio"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
10,horsepower,26.36
31,four,23.46
7,enginesize,19.70
15,sedan,17.09
3,wheelbase,14.13
14,hatchback,13.54
18,rwd,12.82
4,carlength,12.40
5,carwidth,10.12
17,fwd,9.29


In [30]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [31]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.922
Method:                 Least Squares   F-statistic:                     76.52
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           2.03e-85
Time:                        23:27:15   Log-Likelihood:                 320.46
No. Observations:                 205   AIC:                            -574.9
Df Residuals:                     172   BIC:                            -465.3
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           6.744e-17      0.004   1.75e

In [32]:
drop_col=["wheelbase","carlength"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
29,four,19.81
13,sedan,16.84
12,hatchback,12.99
7,enginesize,12.90
3,wheelbase,12.67
4,carlength,11.94
16,rwd,11.39
5,carwidth,9.82
15,fwd,9.21
14,wagon,9.12


In [33]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [34]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     82.55
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           2.51e-87
Time:                        23:27:17   Log-Likelihood:                 320.44
No. Observations:                 205   AIC:                            -578.9
Df Residuals:                     174   BIC:                            -475.9
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           3.621e-17      0.004   9.42e

In [35]:
drop_col=["l","ohcf","spfi"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
27,four,18.22
11,sedan,14.92
10,hatchback,12.09
5,enginesize,11.51
14,rwd,10.89
13,fwd,8.93
12,wagon,8.16
28,six,7.13
8,highwaympg,6.72
3,carwidth,6.17


In [36]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [37]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.934
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     93.13
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           3.36e-90
Time:                        23:27:18   Log-Likelihood:                 320.25
No. Observations:                 205   AIC:                            -584.5
Df Residuals:                     177   BIC:                            -491.5
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           3.751e-17      0.004   9.84e

In [38]:
drop_col=["fwd","rwd","highwaympg"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
24,four,16.45
11,sedan,14.01
10,hatchback,11.51
5,enginesize,10.55
14,rwd,8.50
13,fwd,8.29
12,wagon,7.66
25,six,7.03
8,highwaympg,6.29
3,carwidth,5.71


In [39]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [40]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     103.2
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           4.43e-92
Time:                        23:27:19   Log-Likelihood:                 317.22
No. Observations:                 205   AIC:                            -584.4
Df Residuals:                     180   BIC:                            -501.4
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           3.513e-17      0.004   9.15e

In [41]:
drop_col=["mpfi","carheight","2bbl"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
21,four,16.31
10,sedan,13.46
9,hatchback,11.21
5,enginesize,8.65
11,wagon,7.46
22,six,6.96
20,five,5.37
3,carwidth,5.28
15,mpfi,3.21
12,2bbl,2.95


In [42]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [43]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     117.3
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           2.32e-94
Time:                        23:27:20   Log-Likelihood:                 315.08
No. Observations:                 205   AIC:                            -586.2
Df Residuals:                     183   BIC:                            -513.1
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           3.513e-17      0.004   9.13e

In [44]:
drop_col=["three","4bbl"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
18,four,16.08
9,sedan,12.53
8,hatchback,10.65
4,enginesize,8.33
19,six,6.72
10,wagon,6.37
17,five,5.36
3,carwidth,4.59
11,4bbl,2.73
7,hardtop,2.37


In [45]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [46]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.931
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     130.7
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           2.19e-96
Time:                        23:27:21   Log-Likelihood:                 314.84
No. Observations:                 205   AIC:                            -589.7
Df Residuals:                     185   BIC:                            -523.2
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           3.513e-17      0.004   9.17e

In [47]:
drop_col=["mfi"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
9,sedan,12.51
8,hatchback,10.63
4,enginesize,6.75
17,four,6.64
10,wagon,6.36
3,carwidth,4.20
18,six,4.06
16,five,3.15
7,hardtop,2.37
14,ohc,2.14


In [48]:

X.drop(drop_col,inplace=True,axis=1)
cars.drop(drop_col,inplace=True,axis=1)

In [49]:
import statsmodels.api as sm 
X = sm.add_constant(X)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y,X).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.930
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     136.3
Date:                Thu, 14 Jun 2018   Prob (F-statistic):           8.25e-97
Time:                        23:27:23   Log-Likelihood:                 313.17
No. Observations:                 205   AIC:                            -588.3
Df Residuals:                     186   BIC:                            -525.2
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           3.513e-17      0.004   9.12e

In [50]:
# drop_col=["carwidth","dohcv","wagon"]
vif_cal(input_data=cars, dependent_col="price")


,Var,Vif
9,sedan,12.46
8,hatchback,10.56
16,four,6.63
4,enginesize,6.61
10,wagon,6.32
3,carwidth,4.17
17,six,4.06
15,five,3.15
7,hardtop,2.37
13,ohc,2.14


In [59]:



X=cars
from sklearn.cross_validation import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y, train_size=0.7,test_size=0.3,random_state=100)


In [61]:
import statsmodels.api as sm 
X_train = sm.add_constant(X_train)    # Adding a constant column to our dataframe
# create a first fitted model
lm_4 = sm.OLS(y_train,X_train).fit()
print (lm_4.summary())

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.111e+30
Date:                Thu, 14 Jun 2018   Prob (F-statistic):               0.00
Time:                        23:37:17   Log-Likelihood:                 4895.4
No. Observations:                 143   AIC:                            -9749.
Df Residuals:                     122   BIC:                            -9687.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           8.674e-19   3.08e-17      0.

In [51]:
# drop_col=["carwidth","dohcv","wagon"]
# vif_cal(input_data=cars, dependent_col="price")
